In [1]:
riv_file = "/home/benr/wqmodels/ssm/hyd-test/model_input/ssm_riv.dat"
out_file = "data/fvcom_riv_2014.nc"

import numpy as np
from netCDF4 import Dataset

In [2]:
with open(riv_file) as f:
    # See the FVCOM manual
    inflow_type, point_st_type = np.loadtxt([next(f)], dtype=str, comments='!')
    # The total number of discharge nodes
    num_qs = int(next(f))
    # All the node numbers with discharges
    nodes = np.loadtxt([next(f) for l in range(num_qs)], comments='!', dtype=int)
    # Depth distribution fractions into each node. Skipping the first (node count) column
    vqdist = np.loadtxt([next(f) for l in range(num_qs)])[:,1:]

    num_times = int(next(f))

    # Initialize storage arrays
    times = np.zeros(num_times)
    qs = np.zeros((num_times, num_qs))
    temps = np.zeros((num_times, num_qs))
    salts = np.zeros((num_times, num_qs))

    for t in range(num_times):
        times[t] = float(next(f))
        qs[t,:] = np.loadtxt([next(f)])
        temps[t,:] = np.loadtxt([next(f)])
        salts[t,:] = np.loadtxt([next(f)])

print("Times:", times.shape)
print("Nodes:", nodes.shape)
print("VQdist:", vqdist.shape)
print("Temps:", temps.shape)
temps

Times: (365,)
Nodes: (201,)
VQdist: (201, 10)
Temps: (365, 201)


array([[3.73 , 3.73 , 5.939, ..., 4.5  , 4.5  , 4.5  ],
       [3.73 , 3.73 , 5.939, ..., 4.6  , 4.6  , 4.6  ],
       [3.73 , 3.73 , 5.939, ..., 4.6  , 4.6  , 4.6  ],
       ...,
       [0.05 , 0.05 , 6.784, ..., 7.   , 7.   , 7.   ],
       [0.05 , 0.05 , 6.784, ..., 6.7  , 6.7  , 6.7  ],
       [0.05 , 0.05 , 6.784, ..., 6.3  , 6.3  , 6.3  ]])

In [3]:
cdf = Dataset(out_file, "w")
cdf.inflow_type = inflow_type
cdf.point_st_type = point_st_type

time_dim = cdf.createDimension("time", len(times))
node_dim = cdf.createDimension("node", num_qs)
siglay_dim = cdf.createDimension("siglay", vqdist.shape[1])

time_var = cdf.createVariable("time", "f4", ("time",))
time_var.unit = "hours"
node_var = cdf.createVariable("node", "i4", ("node",))
vqdist_var = cdf.createVariable("vqdist", "f4", ("node","siglay"))
q_var = cdf.createVariable("discharge", "f4", ("time","node"))
temp_var = cdf.createVariable("temp", "f4", ("time","node"))
salt_var = cdf.createVariable("salt", "f4", ("time","node"))

cdf['time'][:] = times
cdf['node'][:] = nodes
cdf['vqdist'][:] = vqdist

cdf['discharge'][:] = qs
cdf['temp'][:] = temps
cdf['salt'][:] = salts

cdf

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    inflow_type: node
    point_st_type: calculated
    dimensions(sizes): time(365), node(201), siglay(10)
    variables(dimensions): float32 time(time), int32 node(node), float32 vqdist(node, siglay), float32 discharge(time, node), float32 temp(time, node), float32 salt(time, node)
    groups: 

In [4]:
cdf.close()